# Transfer learning for COVID-19 Detection in X-Ray Images
## DD2424 Deep Learning - Group Project 

### Read Data

### Proof of concept with simple ML model

### Pretrained VGG-16

### CNN Model from Scratch